# Exploring EBI gene2pheno data for parser

## Downloading data

Latest data (created on the fly) can be downloaded from:
* webpage https://www.ebi.ac.uk/gene2phenotype/download: currently manual clicks only. Can use the "all data" file!
* API urls https://www.ebi.ac.uk/gene2phenotype/api/: `panel/{name}/download/` endpoint. Has "all data" option! Works programmatically and manually

Static release files can be downloaded from the FTP site http://ftp.ebi.ac.uk/pub/databases/gene2phenotype/ (in the G2P_data_downloads subfolder). **These releases will include the "all data" file starting in Aug 2025.** It updates on a monthly basis, according to the [README](http://ftp.ebi.ac.uk/pub/databases/gene2phenotype/README). However, I notice that the FTP site is sometimes slow on my computer (Firefox browser). 


I'm using all data (either downloading the "all data" file or all the panel/subset files). 

## Load data 

In [2]:
## import packages for exploring here

## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pathlib
import pandas as pd
import glob
from pprint import pprint


## for notebook viewing/debugging...
pd.options.display.max_columns = None

In [8]:
## adjust based on where files are stored
base_file_path = pathlib.Path.home().joinpath("Desktop", "EBIgene2pheno_files", 
                                              "From_Website", "2025-07-30")

## uses pathlib's Path.glob, which produces a generator. 
## cast into list so parser code can check if paths were actually matched or not
all_file_paths = list(base_file_path.glob("*.csv"))

### 1 dataframe - compare subset files vs "all" link

These files come from the webpage (manual click downloads) or API.

#### merging subset files

In [38]:
## using generator expression (think list/dict comprehension) within pd.concat to load files 1 at a time
## ingesting all columns as str for now
df_merged = pd.concat((pd.read_csv(f, dtype=str) for f in all_file_paths), ignore_index=True)

## make column names snake-case - usable with itertuples later
df_merged.columns = df_merged.columns.str.replace(" ", "_")

In [39]:
## checking for duplicates - should be since rows in multiple panels

n_duplicates_column_combo = df_merged[df_merged.duplicated(subset=["g2p_id"], keep=False)].shape

n_duplicates_all_columns = df_merged[df_merged.duplicated(keep=False)].shape

## for testing
# n_duplicates_all_columns = (1, 1)

if n_duplicates_column_combo != n_duplicates_all_columns: 
    raise AssertionError("The data format has changed, and the assumptions about duplicates/key columns may " \
                          "no longer hold. Re-explore the data and adjust the parser.")
    
print(n_duplicates_column_combo)

(2066, 23)


In [40]:
df_merged.drop_duplicates(inplace=True, ignore_index=True)

df_merged.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3718 entries, 0 to 3717
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   g2p_id                              3718 non-null   object
 1   gene_symbol                         3718 non-null   object
 2   gene_mim                            3715 non-null   object
 3   hgnc_id                             3718 non-null   object
 4   previous_gene_symbols               3433 non-null   object
 5   disease_name                        3718 non-null   object
 6   disease_mim                         2949 non-null   object
 7   disease_MONDO                       2196 non-null   object
 8   allelic_requirement                 3718 non-null   object
 9   cross_cutting_modifier              464 non-null    object
 10  confidence                          3718 non-null   object
 11  variant_consequence                 3700 non-null   obje

#### "All data" download from webpage 

In [41]:
option_all_path = pathlib.Path.home().joinpath("Desktop", "EBIgene2pheno_files", 
                                              "From_Website", "G2P_all_2025-07-30.csv")

df_option_all = pd.read_csv(option_all_path, dtype=str)
df_option_all.columns = df_option_all.columns.str.replace(" ", "_")

df_option_all.info(memory_usage="deep")
## the numbers are the same as df_merged

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3718 entries, 0 to 3717
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   g2p_id                              3718 non-null   object
 1   gene_symbol                         3718 non-null   object
 2   gene_mim                            3715 non-null   object
 3   hgnc_id                             3718 non-null   object
 4   previous_gene_symbols               3433 non-null   object
 5   disease_name                        3718 non-null   object
 6   disease_mim                         2949 non-null   object
 7   disease_MONDO                       2196 non-null   object
 8   allelic_requirement                 3718 non-null   object
 9   cross_cutting_modifier              464 non-null    object
 10  confidence                          3718 non-null   object
 11  variant_consequence                 3700 non-null   obje

In [42]:
## does the "all" file have duplicates? - no! yay!

df_option_all[df_option_all.duplicated(subset=["g2p_id"], keep=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_support,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,review


In [46]:
## check if these dataframes are identical or not

## first sort them the same way
df_merged.sort_values(by="g2p_id", inplace=True, ignore_index=True)
df_option_all.sort_values(by="g2p_id", inplace=True, ignore_index=True)

## these are identical!!!
df_merged.equals(df_option_all)

True

#### "All data" download from API url

In [44]:
## pandas downloading file from URL: works, a little slow

df_url_all = pd.read_csv("https://www.ebi.ac.uk/gene2phenotype/api/panel/all/download/", 
                         dtype=str)

In [45]:
df_url_all.columns = df_url_all.columns.str.replace(" ", "_")

df_url_all.info(memory_usage="deep")
## the numbers are the same as df_merged and df_option_all

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3718 entries, 0 to 3717
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   g2p_id                              3718 non-null   object
 1   gene_symbol                         3718 non-null   object
 2   gene_mim                            3715 non-null   object
 3   hgnc_id                             3718 non-null   object
 4   previous_gene_symbols               3433 non-null   object
 5   disease_name                        3718 non-null   object
 6   disease_mim                         2949 non-null   object
 7   disease_MONDO                       2196 non-null   object
 8   allelic_requirement                 3718 non-null   object
 9   cross_cutting_modifier              464 non-null    object
 10  confidence                          3718 non-null   object
 11  variant_consequence                 3700 non-null   obje

In [48]:
## check if sliced dataframe is identical to others or not

## first sort them the same way
df_url_all.sort_values(by="g2p_id", inplace=True, ignore_index=True)

## these are identical!!!
## meaning this also doesn't have duplicates
df_option_all.equals(df_url_all)

True

<div class="alert alert-block alert-success">


Conclusions:
* "all data" file does have all data, w/o duplicates. So it is nice to use!
* Using API link to retrieve "all data" file is the easiest to set up common ingest pipeline to use

### original: 1 dataframe for each subset

In [4]:
## Construct df for each file

cancer = pd.read_csv(base_file_path.joinpath('G2P_Cancer_2025-03-13.csv'), 
                      dtype={"gene mim": str, "hgnc id": str, "disease mim": str})
cardiac = pd.read_csv(base_file_path.joinpath('G2P_Cardiac_2025-03-13.csv'), 
                      dtype={"gene mim": str, "hgnc id": str, "disease mim": str})
developmental = pd.read_csv(base_file_path.joinpath('G2P_DD_2025-03-13.csv'), 
                      dtype={"gene mim": str, "hgnc id": str, "disease mim": str})
eye = pd.read_csv(base_file_path.joinpath('G2P_Eye_2025-03-13.csv'), 
                      dtype={"gene mim": str, "hgnc id": str, "disease mim": str})
hearing = pd.read_csv(base_file_path.joinpath('G2P_Hearing loss_2025-03-13.csv'), 
                      dtype={"gene mim": str, "hgnc id": str, "disease mim": str})
skeletal = pd.read_csv(base_file_path.joinpath('G2P_Skeletal_2025-03-13.csv'), 
                      dtype={"gene mim": str, "hgnc id": str, "disease mim": str})
skin = pd.read_csv(base_file_path.joinpath('G2P_Skin_2025-03-13.csv'), 
                      dtype={"gene mim": str, "hgnc id": str, "disease mim": str})

## saving for possible future use
# unique_filename_substring = [
#     'Cancer',
#     'Cardiac',
#     'DD',
#     'Eye',
#     'Hearing loss',
#     'Skeletal',
#     'Skin'
# ]

## Code chunks to explore individual files

In [5]:
## checking that each file's columns are the same: they are
cancer.columns == skin.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [6]:
cardiac.columns

Index(['g2p id', 'gene symbol', 'gene mim', 'hgnc id', 'previous gene symbols',
       'disease name', 'disease mim', 'disease MONDO', 'allelic requirement',
       'cross cutting modifier', 'confidence', 'variant consequence',
       'variant types', 'molecular mechanism',
       'molecular mechanism categorisation', 'molecular mechanism evidence',
       'phenotypes', 'publications', 'panel', 'comments',
       'date of last review'],
      dtype='object')

Reviewing individual dataframes, values within each column

In [7]:
skin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   g2p id                              607 non-null    object
 1   gene symbol                         607 non-null    object
 2   gene mim                            607 non-null    object
 3   hgnc id                             607 non-null    object
 4   previous gene symbols               554 non-null    object
 5   disease name                        607 non-null    object
 6   disease mim                         475 non-null    object
 7   disease MONDO                       166 non-null    object
 8   allelic requirement                 607 non-null    object
 9   cross cutting modifier              133 non-null    object
 10  confidence                          607 non-null    object
 11  variant consequence                 607 non-null    object

In [8]:
## was looking for duplicate records within 1 dataframe?

skin[skin.duplicated(subset=['g2p id'], keep=False)].sort_values(by=['gene symbol', 'disease name'])

skin[skin.duplicated(subset=['gene symbol', 'disease name', 'allelic requirement', 'molecular mechanism'], keep=False)].sort_values(by=['gene symbol', 'disease name'])

,g2p id,gene symbol,gene mim,hgnc id,previous gene symbols,disease name,disease mim,disease MONDO,allelic requirement,cross cutting modifier,confidence,variant consequence,variant types,molecular mechanism,molecular mechanism categorisation,molecular mechanism evidence,phenotypes,publications,panel,comments,date of last review


,g2p id,gene symbol,gene mim,hgnc id,previous gene symbols,disease name,disease mim,disease MONDO,allelic requirement,cross cutting modifier,confidence,variant consequence,variant types,molecular mechanism,molecular mechanism categorisation,molecular mechanism evidence,phenotypes,publications,panel,comments,date of last review


In [462]:
skin["date of last review"].nunique()

341

In [ ]:
## useful function
def check_if_contains(df, column_name, patterns):
    for i in patterns:
        temp = df[df[column_name].str.contains(pat=i, na=False)]
        if temp.size > 0:
            print(f'"{i}"')
            print(temp.shape)

In [463]:
## check for ";", ":", "_", " "
# check_if_contains(skin, "publications", ["PMID", ":", "_"])
# skin[skin["disease mim"].str.contains(":", na=False)]
            
# skin["comments"].nunique()
skin["date of last review"].unique()[0:100]

array(['2024-08-20 14:13:58+00:00', '2017-09-01 16:19:16+00:00',
       '2025-01-15 11:51:09+00:00', '2017-09-01 16:19:15+00:00',
       '2023-05-24 09:07:26+00:00', '2025-01-27 22:44:48+00:00',
       '2025-01-27 16:20:20+00:00', '2018-01-21 21:15:11+00:00',
       '2025-01-27 15:06:26+00:00', '2025-01-21 21:25:35+00:00',
       '2025-01-27 16:10:36+00:00', '2022-09-08 10:23:08+00:00',
       '2025-01-28 23:05:54+00:00', '2010-01-01 00:00:00+00:00',
       '2024-08-20 14:10:55+00:00', '2022-08-03 13:41:12+00:00',
       '2025-01-28 23:06:55+00:00', '2019-09-16 15:47:08+00:00',
       '2025-01-27 14:32:43+00:00', '2025-01-27 14:33:14+00:00',
       '2025-01-20 23:47:41+00:00', '2025-01-21 13:09:11+00:00',
       '2022-08-03 13:34:46+00:00', '2023-05-24 09:07:27+00:00',
       '2015-07-22 16:14:25+00:00', '2024-12-13 14:31:01+00:00',
       '2025-01-20 22:48:05+00:00', '2018-07-05 15:56:34+00:00',
       '2024-12-13 11:11:03+00:00', '2024-12-23 20:57:52+00:00',
       '2025-01-16 17:15:

In [451]:
# eye[eye["gene mim"].isna()]

skin[skin["disease mim"].isna() & skin["disease MONDO"].isna()].shape

# cancer[cancer["comments"].notna()]

(62, 21)

In [353]:
hearing

,g2p id,gene symbol,gene mim,hgnc id,previous gene symbols,disease name,disease mim,disease MONDO,allelic requirement,cross cutting modifier,confidence,variant consequence,variant types,molecular mechanism,molecular mechanism categorisation,molecular mechanism evidence,phenotypes,publications,panel,comments,date of last review
0,G2P03582,MYO6,600970,7605,DFNA22; DFNB37; KIAA0389,MYO6-related nonsyndromic genetic hearing loss,NaN,NaN,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,splice_donor_variant; splice_acceptor_variant;...,undetermined,inferred,NaN,NaN,18348273; 23485424; 25999546; 12687499; 24105371,Hearing loss,NaN,2024-11-28 14:52:17+00:00
1,G2P03583,MYO6,600970,7605,DFNA22; DFNB37; KIAA0389,MYO6-related nonsyndromic genetic hearing loss,NaN,NaN,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,splice_donor_variant; splice_acceptor_variant;...,undetermined,inferred,NaN,NaN,18348273; 23485424; 25999546; 24105371,Hearing loss,NaN,2024-11-28 14:47:17+00:00


In [357]:
hearing['publications'].unique()

array(['18348273; 23485424; 25999546; 12687499; 24105371',
       '18348273; 23485424; 25999546; 24105371'], dtype=object)

## Notes on parsing data to create documents 

### Gene subject section

- "gene symbol": str. "HGNC-assigned gene symbol" (according to DataDownloadFormat txt file)
  - DOC TRANSFORM: hgnc_symbol
- "gene mim": OMIM ID for gene. NodeNorm does recognize these. Some files have a NA.
  - DF READ: force dtype to be str
  - DF TRANSFORM: 
    - add current Translator prefix
  - DOC TRANSFORM: omim
    - CHECK for NA. Only assign field if not NA  
- "hgnc id"
  - DF READ / DF TRANSFORM: same as "gene mim"
  - DOC TRANSFORM: hgnc
- "previous gene symbols": "; "-delimited.
  - DOC TRANSFORM:
    - CHECK for NA. Only assign field if not NA
    - Turn into array of str: ";"-split and then strip whitespace 

### Disease object section

- "disease name": in DD file, I found two cases where the name seemed to be "; "-delimited. Going to keep as str though, because it's used as node name. 
  - DF TRANSFORM:
    - strip whitespace (noticed a case in skeletal)
  - DOC TRANSFORM: name
- "disease mim": OMIM ID for disease. But there are some Orphanet IDs mixed in (raw data prefix is "Orphanet:")
  - DF READ: force dtype to be str
  - DF TRANSFORM: add/replace current Translator prefix
  - DOC TRANSFORM: omim, orphanet
    - CHECK for NA (many rows are). Only assign field if not NA
    - CHECK for "orphanet:": if yes, assign to orphanet field instead
- "disease MONDO": MONDO ID for disease (already Translator-formatted CURIE) 
  - DOC TRANSFORM: mondo
    - CHECK for NA (many rows are). Only assign field if not NA

### Association section

- "g2p id": resource's unique stable ID for each record/row (according to DataDownloadFormat txt file, G2P webpage for single record)
  - DF TRANSFORM:
    - create new column, generating G2P webpage urls for each record like https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03386. this may be brittle/work to maintain, but I know UI really wants stuff like this
  - DOC TRANSFORM: g2p_record_id, g2p_record_url
- "allelic requirement": required genotype (according to G2P webpage for single record). str (categorical). [G2P website](https://www.ebi.ac.uk/gene2phenotype/about/terminology) says these are synonyms of HPO "mode of inheritance terms", which seem to be from the [inheritance part of the ontology](https://hpo.jax.org/browse/term/HP:0000005). I've included a mapping table in reference section.
- "cross cutting modifier": additional info relevant to gene-disease inheritance (according to G2P webpage for single record). Can be "; "-delimited, otherwise would be categorical. [G2P website](https://www.ebi.ac.uk/gene2phenotype/about/terminology) says these are HPO inheritance qualifier terms "when available" ([this part of ontology](https://hpo.jax.org/browse/term/HP:0034335)). I didn't try mapping these.
  - DOC TRANSFORM: cross_cutting_modifiers (plural)
    - CHECK for NA (many rows are). Only assign field if not NA
    - Turn into array of str: ";"-split and then strip whitespace 
- "confidence": confidence that the association is real (according to G2P webpage for panel). [G2P website](https://www.ebi.ac.uk/gene2phenotype/about/terminology) has definitions of terms. str (categorical)
- "variant consequence": consequences of reported variants on product (protein or RNA) (according to G2P webpage for single record). Can be "; "-delimited, otherwise would be categorical. [G2P website](https://www.ebi.ac.uk/gene2phenotype/about/terminology) has definitions of terms and SO mappings. 
  - DOC TRANSFORM: variant_consequences (plural)
    - CHECK for NA. Only assign field if not NA
    - Turn into array of str: ";"-split and then strip whitespace 
- "variant types": associated with gene-disease pair (according to G2P webpage for single record). Can be "; "-delimited, otherwise would be categorical. [G2P website](https://www.ebi.ac.uk/gene2phenotype/about/terminology) has definitions of terms and SO mappings (large table!). 
  - DOC TRANSFORM:
    - CHECK for NA (many rows are, sometimes). Only assign field if not NA
    - Turn into array of str: ";"-split and then strip whitespace 
- "molecular mechanism": mechanism of how gene's variants causes disease (according to G2P webpage for single record). str (categorical). [G2P website](https://www.ebi.ac.uk/gene2phenotype/about/terminology) has definitions of terms.
- "molecular mechanism categorisation": seems to say how molecular mechanism was decided. Shows up as "support" on G2P webpage for single record. possible values: "inferred" or "evidence". str (categorical). 
- "molecular mechanism evidence": complicated structure. Looks like " & "-delimited, then primary_key (publication ID) -> object (diff key: value pairs split by "; ", values can be ", "-delimited lists). 
  - DOC TRANSFORM:
    - CHECK for NA (many rows are). Only assign field if not NA
    - Turn into array of str: "&"-split and then strip whitespace. Not going to do more parsing for now.  
- "phenotypes": HPO IDs (already Translator-formatted CURIE) for phenotypes reported by the publications (according to G2P webpage for single record). Can be "; "-delimited
  - based on G2P website, these are organized/assigned to specific gene-disease records...so I'm keeping as a part of the gene-disease assocation, rather than creating diff objects for gene-phenotype (if I did this, it would be a separate function)
  - DOC TRANSFORM:
    - CHECK for NA (many rows are, sometimes). Only assign field if not NA
    - Turn into array of str: ";"-split and then strip whitespace 
- "publications": these are PMIDs with no prefix (based on G2P webpage for single record). Can be "; "-delimited
  - DOC TRANSFORM: pmids
    - CHECK for NA (many rows are, sometimes). Only assign field if not NA
    - Turn into array of str: ";"-split and then strip whitespace 
- "panel": the G2P panels this record is assigned to (according to DataDownloadFormat txt file). Can be "; "-delimited, otherwise would be categorical.
  - DOC TRANSFORM: g2p_panels
    - map (str.replace) values to match G2P webpage for single record
      - "DD" = "Developmental disorders"
      - add " disorders" to end: "Cancer", "Cardiac", "Eye", "Skeletal", "Skin"
      - keep as-is: "Hearing loss"
    - Turn into array of str: ";"-split and then strip whitespace 
- "comments": additional comments from curation team (according to DataDownloadFormat txt file). Appears to be free-text (sometimes short, sometimes very long)
  - DF TRANSFORM:
    - strip whitespace
  - DOC TRANSFORM: curator_comments
    - CHECK for NA (many rows are, sometimes). Only assign field if not NA
- "date of last review":
  - DF TRANSFORM:
    - optional: turn dtype into datetime https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html#pandas.to_datetime


__Allelic requirement terms__

| G2P | HPO name | HPO ID | Exact synonym? | 
| :- | :- | :- | :- |
| biallelic_autosomal | Autosomal recessive inheritance | HP:0000007 | Yes |
| monoallelic_autosomal | Autosomal dominant inheritance | HP:0000006 | Yes |
| biallelic_PAR | Pseudoautosomal recessive inheritance | HP:0034341 | Yes |
| monoallelic_PAR | Pseudoautosomal dominant inheritance | HP:0034340 | Yes |
| mitochondrial | Mitochondrial inheritance | HP:0001427 | Yes |
| monoallelic_Y_hemizygous | Y-linked inheritance | HP:0001450 | Yes |
| monoallelic_X | X-linked inheritance | HP:0001417 | No (my guess) |
| monoallelic_X_hemizygous | X-linked recessive inheritance | HP:0001419 | No (my guess) |
| monoallelic_X_heterozygous | X-linked dominant inheritance | HP:0001423 | No (my guess) |

---

__Not doing right now: optional making new columns/fields__

Would do value-mapping using static dicts? Which would be brittle/work to maintain

* "allelic requirement": create new columns for HPO mapping IDs/labels (reference below)
* "variant consequence": create new column(s) for SO mapping (value is ID or labels, or a list of dict for ID/label pairs)
* "variant types": create new column(s) for SO mapping

## Issues merging data into 1 dataframe

If I loaded all data into 1 dataframe, there will likely be duplicate rows because the gene-disease pair is in several panels (aka disease falls into multiple categories). One proof of this is `panel` being a "; "-delimited string.

Drop duplicates will be needed. 

However, what if other column values differ between files (like "; "-delimited strings having a diff order of elements)? Especially wondering about `panel`. 

Exploring this:
* Add all dataframes together into a big dataframe
* how many duplicates are there based on the columns that matter for unique row: 'g2p id'
  * other important set: 'gene symbol', 'disease name', 'allelic requirement', 'molecular mechanism'
* VS how many duplicates are there considering all columns? 

If the duplicate numbers differ between the two, then some other column's values are contributing to "unique" rows between diff files. (probably everything duplicate count < subset duplicate count)
* If the problem is "; "-delimited string order, then can reorder them (split, sort, join. Can you do this column-wise rather than iterating?). Then can look for duplicate rows.

If the numbers are the same, then we're good! No other columns matter for unique row, and the duplicates can be dropped (based on all columns). 

In [ ]:
everything = pd.concat(objs=[cancer, cardiac, developmental, eye, hearing, skeletal, skin], ignore_index=True)

In [10]:
everything.shape

cancer.shape[0] + cardiac.shape[0] + \
developmental.shape[0] + eye.shape[0] + \
hearing.shape[0] + skeletal.shape[0] + \
skin.shape[0]

(4721, 21)

4721

In [11]:
everything[everything.duplicated(subset=['g2p id'], keep=False)].sort_values(by=['gene symbol', 'disease name']).shape


everything[everything.duplicated(subset=['gene symbol', 'disease name', 'allelic requirement', 'molecular mechanism'], keep=False)].sort_values(by=['gene symbol', 'disease name']).shape

everything[everything.duplicated(keep=False)].shape

(1935, 21)

(1935, 21)

(1935, 21)

In [12]:
everything[everything["g2p id"] == "G2P00650"]

,g2p id,gene symbol,gene mim,hgnc id,previous gene symbols,disease name,disease mim,disease MONDO,allelic requirement,cross cutting modifier,confidence,variant consequence,variant types,molecular mechanism,molecular mechanism categorisation,molecular mechanism evidence,phenotypes,publications,panel,comments,date of last review
839,G2P00650,AAAS,605378,13666,NaN,AAAS-related chalasia-addisonianism-alacrima s...,231550,NaN,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,HP:0002459; HP:0011463; HP:0003487; HP:0001278...,11062474; 11701718; 18628786; 11159947; 15173230,DD; Eye; Skin,NaN,2025-01-27 14:40:58+00:00
3104,G2P00650,AAAS,605378,13666,NaN,AAAS-related chalasia-addisonianism-alacrima s...,231550,NaN,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,HP:0002459; HP:0011463; HP:0003487; HP:0001278...,11062474; 11701718; 18628786; 11159947; 15173230,DD; Eye; Skin,NaN,2025-01-27 14:40:58+00:00
4198,G2P00650,AAAS,605378,13666,NaN,AAAS-related chalasia-addisonianism-alacrima s...,231550,NaN,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,HP:0002459; HP:0011463; HP:0003487; HP:0001278...,11062474; 11701718; 18628786; 11159947; 15173230,DD; Eye; Skin,NaN,2025-01-27 14:40:58+00:00


So...the numbers are the same! This means duplicate records/rows from diff files are completely identical, and can be dropped easily (based on all columns)

## Other parser code ideas

* each row => turn into 1 association object
  * document/object MUST have `_id` key (primary ID for BioThings database - ElasticSearch / mongo DB?) 
    * use 'g2p id' for `_id`
    * unique 'gene symbol', 'disease name', 'allelic requirement', 'molecular mechanism' combo
    * "G2P records are Locus-Genotype-Mechanism-Disease-Evidence (LGMDE) threads describing gene-disease associations" -> this is what a unique row is from resource POV (ref: homepage table "Total LGMDE Records" info (i) button)
* sometimes disease doesn't have any IDs for it => still make document/object but won't use in Translator unless it has ID
  * all hearing records (2) have this problem.
* str split method will create single-element arrays if no delimiter found
* parser feeds into upload step -> function should be generator to iterate through all stuff -> for each, yield the document(s) as dict/json

Confidence is important to filter on for Translator! => will still make document/object but don't use! 
* don't use "limited", "disputed", "refuted": Translator doesn't handle negation ("refuted") or low/conflicting evidence ("disputed" or "limited")